In [27]:
# convert the original popper ILP to relational datasets
import sys
import os
import re
import numpy as np
import pandas as pd

output_folder = "relational"

In [28]:
# first do the labels

labels = {"train_id": [], "class": []}
with open("originalILP/exs.pl",'r') as f:
    lines = f.readlines()
    
    pattern = r'(pos|neg)\(f\(t(\d+)\)\)'
    
    for line in lines:
        match = re.match(pattern, line)
        if match:
            labels["train_id"].append("t"+match.group(2))
            labels["class"].append(match.group(1))

df = pd.DataFrame(labels)
df.to_csv(os.path.join(output_folder, "train.csv"), index=False)

In [29]:
# bk file

kb = {}
with open("originalILP/bk.pl",'r') as f:
    lines = f.readlines()

    pattern = r"(\w+)\(([^)]+)\)"

    for line in lines:
        match = re.match(pattern, line)
        if match:
            predicate = match.group(1)
            args = match.group(2).split(',')
            if predicate not in kb:
                kb[predicate] = []
            kb[predicate].append(args)


# turn the kb into dataframes

for predicate in kb:
    if predicate == "has_car":
        columns = ["train_id","car_id"]
    elif predicate == "has_load":
        columns = ["car_id","load_id"]
    elif predicate in ["short","long","two_wheels","three_wheels","roof_open","roof_closed"]:
        columns = ["car_id"]
    elif predicate in ["zero_load","one_load","two_load","three_load","circle","triangle","square"]:
        columns = ["load_id"]
    else:
        continue
    df = pd.DataFrame(kb[predicate], columns = columns)
    df.to_csv(f"{output_folder}/{predicate}.csv", index = False)